In [10]:
# testing

# tests for getMarketData

# len(my_list) == len(set(my_list))  # if true, the set has only unique values

In [11]:
import requests
import string 
import pandas as pd
import time

In [12]:
# int -> response
# returns the web response from the PI API
def getMarketRespose(market_id):
    api_url = "https://www.predictit.org/api/marketdata/markets/"
    joined_url = api_url + str(market_id)
    
    response = requests.get(joined_url)  # fetch market data
    if("null" == response.text):
        print("Bad predictItID -- No market with that ID")
    return response

In [13]:
# None -> list
# returns a list of all open markets 
def getAllOpenMarketIDs():
    # commenting out so I'm not spamming the API
    # api_url = "https://www.predictit.org/api/marketdata/all/"
    # 
    # response = requests.get(api_url)  # fetch market data
    if(response.status_code != 200):
        print("Please wait...")
        time.sleep(3)
        print("Please try again now. If this issue persists, the API access may have changed.")

    xml_response = response.json()
    all_markets_df = pd.DataFrame.from_dict(xml_response)


    market_id_list = []  
    
    for market_index in range(len(all_markets_df)):
        # print(all_markets_df["markets"][market_index]["id"])
        market_id = all_markets_df["markets"][market_index]["id"]
        market_id_list.append(market_id)

    return market_id_list

In [14]:
# TODO: change to reference the all dataframe so I'm not making a ton of requests
# 2.7 kB individual market vs 453 kB for all 
# int -> dataframe
def getMarketDataFrame(market_id):
    response = getMarketRespose(market_id)  
    xml_response = response.json()
    
    market_df = pd.DataFrame.from_dict(xml_response)
    return market_df

In [15]:
# int -> print data 
def printMarketData(market_id):
    # market_df = getMarketDataFrame(market_id)  # whole market, including 
    
    try:
        print("Market:\t", market_df["shortName"][0])  # every market has a shortname
    except KeyError:
        print("That market doesn't have a key \"shortName\"")
        return
    tabspacing = "\t\t"
    print("name", tabspacing, "Last Close Price")
    
    
    for contract_index in range(0, len(market_df)):
        all_contracts = market_df["contracts"][contract_index]
        print(all_contracts["name"], end=tabspacing)
        print(all_contracts["lastClosePrice"], end="\t")
        print()  # newline


In [16]:
# int -> print data 
def getBetterMarketDF(market_id):
    market_df = getMarketDataFrame(market_id)  # whole market, including 
    
    try:
        print("Market:\t", market_df["shortName"][0])
    except KeyError:
        print("That market doesn't have a key \"shortName\"")
        return
    tabspacing = "\t\t"
    print("name", tabspacing, "Last Close Price")
    
    
    for contract_index in range(0, len(market_df)):
        all_contracts = market_df["contracts"][contract_index]
        print(all_contracts["name"], end=tabspacing)
        print(all_contracts["lastClosePrice"], end="\t")
        print()  # newline







In [17]:
# TODO? image grabbing function
# from PIL import Image
# url = market_df["image"][0]
# im = Image.open(requests.get(url, stream=True).raw)


In [18]:
# request data for entire market 
# None -> response
def getAllMarketsResponse():
    api_url = "https://www.predictit.org/api/marketdata/all/"
    response = requests.get(api_url)  # fetch market data
    if(response.status_code != 200):
        print("Please wait for API cooldown")
        
        for second in  list(range(10))[::-1]:
            time.sleep(1)
            print(second)
        print("Please try again now. If the issue persists, API access may have changed.")
        return None
    return response

In [67]:
# None -> list of all the open markets
def getAllMarketsList(market_data=False):
    all_markets_response = getAllMarketsResponse()
    return all_markets_response.json()["markets"]

def fetchIfNoMarketData(market_data=False):
    if market_data is False:
        print("Now fetching market data")
        market_data = getAllMarketsList()
        return market_data
    else:
        return market_data
    
    
    
def getIdIndexList( market_data=False):
    market_data = fetchIfNoMarketData(market_data)
    id_index_list = []  # 
    
    for mkt_index in range(len(market_data)):
        id_index_list.append(market_data[mkt_index]["id"])
        # print(id_index_list)
    return id_index_list

# returns all the contracts of a particular market based on the index
# a market index of 0 is the most recent and the largest valid value is the oldest open market
def getContractsDataFrameFromIndex( mkt_index, market_data=False):
    market_data = fetchIfNoMarketData(market_data)
    
    # this references the first market in the list of all markets  ????
    market_df = pd.DataFrame.from_dict(market_data[ mkt_index ]["contracts"])
    return market_df


def getIndexFromMarketId( mkt_id, market_data=False):
    market_data = fetchIfNoMarketData(market_data)
    market_index_list = getIdIndexList(market_data=market_data)
    try:
        return market_index_list.index(mkt_id)
    except ValueError:
        print(f"There exists no open contracts with id {mkt_id}.")
        return None
    
# returns all the contracts of a particular market based on ID
def getContractsDataFrame( mkt_id, market_data=False):
    market_data = fetchIfNoMarketData(market_data)
    # use the mkt_id to find the market index
    
    # TD # market_index_list = getIdIndexList(market_data=market_data)  # obsoleted, delete tests pass for gIndexfmMktId
    market_index = getIndexFromMarketId(mkt_id, market_data=market_data)
    
    # TD
    # try:
    #     market_df = getContractsDataFrameFromIndex( market_index, market_data=market_data)
    # except ValueError:
    #     print(f"There exists no open contracts with id {mkt_id}.")
    #     return None
    # /TD
    
    return getContractsDataFrameFromIndex(market_index, market_data=market_data)

# adds the word market to a string; used to label market data for concatanation in contracts DataFrame
def concatMarketString(string):
    return "market"+string.capitalize()

# append "market" in front of each column in a dataframe if it doesn't already have the label
# df -> df
def labelMarket(market_df):
    if(market_df.columns[0][0:6] == "market"):  # if marketId is already labeled, they're every column is already labeled (hopefully)
        return market_df
    else:
        return market_df.rename(concatMarketString, axis="columns")

# """
# TODO
# [ ] prepend the general market data frame with the contract specific dataframe
# [ ] look at sensible plotting options. Format data in a way that's ammenable to those options
# [ ] make csv saving job so that I don't need to worry about 
# """


In [20]:
# TODO 
#class MarketSnapshot( market_data=):
    # datetime retrieved
    # retrieve market data if not passed
    
    # put all previously created funcitons here 

In [86]:
# concatonate market data frame attribute to each contract
#whole_market_data = getAllMarketsList(market_data=whole_market_data)

# #display(getContractsDataFrameFromIndex( mkt_index, market_data=whole_market_data))
# 
# # mkt_id = 7061  # Yellen num votes
# mkt_id = 6944  # SoS
# 

def getFullMarketInfoByID(mkt_id, market_data=market_df):
    # market df
    mkt_index = getIndexFromMarketId(mkt_id, whole_market_data)
    market_df = pd.DataFrame.from_dict(whole_market_data[mkt_index])
    market_df = labelMarket(market_df)

    # contract df 
    contracts_df = getContractsDataFrame(mkt_id, market_data=whole_market_data)
    
    # combine dfs
    return market_df.join(contracts_df)
    

In [94]:
getFullMarketInfoByID(6944, market_data=whole_market_data)

,marketId,marketName,marketShortname,marketImage,marketUrl,marketContracts,marketTimestamp,marketStatus,id,dateEnd,...,name,shortName,status,lastTradePrice,bestBuyYesCost,bestBuyNoCost,bestSellYesCost,bestSellNoCost,lastClosePrice,displayOrder
0,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24143, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24143,2021-02-15T23:59:00,...,Antony Blinken,Blinken,Open,0.96,0.97,0.04,0.96,0.03,0.97,0
1,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24134, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24134,2021-02-15T23:59:00,...,Mike Pompeo,Pompeo,Open,0.02,0.02,0.99,0.01,0.98,0.01,0
2,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24135, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24135,2021-02-15T23:59:00,...,Nikki Haley,Haley,Open,0.01,0.01,NaN,NaN,0.99,0.01,0
3,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24136, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24136,2021-02-15T23:59:00,...,Tom Cotton,Cotton,Open,0.01,0.01,NaN,NaN,0.99,0.01,0
4,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24138, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24138,2021-02-15T23:59:00,...,Chris Coons,Coons,Open,0.01,0.01,NaN,NaN,0.99,0.01,0
5,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24139, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24139,2021-02-15T23:59:00,...,Chris Murphy,Murphy,Open,0.01,0.01,NaN,NaN,0.99,0.01,0
6,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24140, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24140,2021-02-15T23:59:00,...,Susan Rice,Rice,Open,0.01,0.01,NaN,NaN,0.99,0.01,0
7,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24141, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24141,2021-02-15T23:59:00,...,Mike Donilon,M. Donilon,Open,0.01,0.01,NaN,NaN,0.99,0.01,0
8,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24142, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24142,2021-02-15T23:59:00,...,Hillary Clinton,Clinton,Open,0.01,0.01,NaN,NaN,0.99,0.01,0
9,6944,Who will be the Senate-confirmed Secretary of ...,Secretary of State on Feb. 15?,https://az620379.vo.msecnd.net/images/Markets/...,https://www.predictit.org/markets/detail/6944/...,"{'id': 24144, 'dateEnd': '2021-02-15T23:59:00'...",2021-01-22T13:47:09.7276261,Open,24144,2021-02-15T23:59:00,...,Marco Rubio,Rubio,Open,0.01,0.01,NaN,NaN,0.99,0.01,0


,id,dateEnd,image,name,shortName,status,lastTradePrice,bestBuyYesCost,bestBuyNoCost,bestSellYesCost,bestSellNoCost,lastClosePrice,displayOrder
0,22395,NA,https://az620379.vo.msecnd.net/images/Contract...,GOP by 10.5% or more,GOP by 10.5%+,Open,0.02,0.03,0.98,0.02,0.97,0.01,0
1,22404,NA,https://az620379.vo.msecnd.net/images/Contract...,GOP by 9.0% - 10.5%,GOP by 9% - 10.5%,Open,0.01,0.01,NaN,NaN,0.99,0.01,1
2,22402,NA,https://az620379.vo.msecnd.net/images/Contract...,GOP by 7.5% - 9.0%,GOP by 7.5% - 9%,Open,0.01,0.01,NaN,NaN,0.99,0.01,2
3,22400,NA,https://az620379.vo.msecnd.net/images/Contract...,GOP by 6.0% - 7.5%,GOP by 6% - 7.5%,Open,0.01,0.01,NaN,NaN,0.99,0.01,3
4,22394,NA,https://az620379.vo.msecnd.net/images/Contract...,GOP by 4.5% - 6.0%,GOP by 4.5% - 6%,Open,0.01,0.01,NaN,NaN,0.99,0.01,4
5,22392,NA,https://az620379.vo.msecnd.net/images/Contract...,GOP by 3.0% - 4.5%,GOP by 3% - 4.5%,Open,0.01,0.01,NaN,NaN,0.99,0.01,5
6,22391,NA,https://az620379.vo.msecnd.net/images/Contract...,GOP by 1.5% - 3.0%,GOP by 1.5% - 3%,Open,0.01,0.01,NaN,NaN,0.99,0.01,6
7,22390,NA,https://az620379.vo.msecnd.net/images/Contract...,"GOP, less than 1.5%",GOP by 1.5%-,Open,0.01,0.01,NaN,NaN,0.99,0.01,7
8,22396,NA,https://az620379.vo.msecnd.net/images/Contract...,"Dems, less than 1.5%",Dems by 1.5%-,Open,0.01,0.01,NaN,NaN,0.99,0.01,8
9,22397,NA,https://az620379.vo.msecnd.net/images/Contract...,Dems by 1.5% - 3.0%,Dems by 1.5% - 3%,Open,0.01,0.01,NaN,NaN,0.99,0.01,9


In [ ]:
# float, float -> bool
def inSweetSpot( value, sweet_spot=0.95, threshold=0.05 ):
    if( sweet_spot - threshold <= value <= sweet_spot+threshold):
        return True
    else:
        return False


In [ ]:
# identify markets with negative risk

whole_market_data = getAllMarketsList()  # toggle for data retrieval
senators_vote = getContractsDataFrame(7054, market_data=whole_market_data)

filtered_buy_no = senators_vote.filter(["name", "bestBuyNoCost"])

display(filtered_buy_no)
cost = filtered_buy_no["bestBuyNoCost"].sum()
print("total cost to buy one of each no contract: ", cost)

winning_contract = 1  # 51 or 52 vote yes or something like that 

for contract in range(len(filtered_buy_no)):
    filtered_buy_no.at[contract, "bestBuyNoCost"] = 1
    
filtered_buy_no.at[winning_contract, "bestBuyNoCost"] = 0

display(filtered_buy_no)
#print(zero_scenario)